# Data Engineer Challenge

## Introducción
En este notebook, se ofrece una solución al desafío de procesamiento de datos con Python. El propósito es examinar un conjunto de datos de tweets y abordar diversas preguntas mediante dos enfoques: uno enfocado en optimizar el tiempo de ejecución y otro en reducir el consumo de memoria.

## Estructura del Proyecto
La solución se organiza en diferentes funciones, cada una en su archivo correspondiente dentro de la carpeta `src`. A continuación, se presentan tres preguntas, cada una con dos enfoques.


Instalar paquetes necesarios

In [105]:
%pip install memory-profiler

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Configuramos la ruta del archivo

In [106]:
file_path = r"C:\temp\Challenge_DATA\tweets.json\farmers-protest-tweets-2021-2-4.json"

Carga de Librerías

In [107]:
import json
import time
from datetime import datetime, date
import re
from collections import defaultdict, Counter
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor
from memory_profiler import memory_usage


Definimos una función para analizar el performance

In [108]:
def measure_function(func, *args):
    """Mide el tiempo de ejecución y el uso de memoria de una función."""
    start_time = time.time()
    result = func(*args)
    end_time = time.time()

    mem_usage = memory_usage((func, args), max_usage=True)

    return result, end_time - start_time, mem_usage

Funciones
1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.


Código que no considera optimizar recursos de procesamiento

In [109]:
def q1_deficiente(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Devuelve las 10 fechas con más tweets y el usuario que más tweets publicó en esas fechas.

    Parámetros:
    - file_path: Ruta del archivo JSON que contiene los tweets.

    Retorno:
    - Lista de tuplas donde cada tupla contiene una fecha y el nombre del usuario que más tweets publicó
      en esa fecha.
    """

    # Cargar todos los tweets en memoria
    tweets = []

    with open(file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            tweet = json.loads(line)
            tweets.append(tweet)

    # Inicializa contadores de tweets por fecha y usuario
    date_tweet_count = defaultdict(int)
    date_user_tweets = defaultdict(lambda: defaultdict(int))

    # Procesar los tweets
    for tweet in tweets:
        date_str = tweet["date"][:10]  # Extrae la parte de la fecha
        username = tweet["user"]["username"]  # Obtiene el nombre de usuario

        # Incrementar el contador para la fecha y el usuario
        date_tweet_count[date_str] += 1
        date_user_tweets[date_str][username] += 1

    # Lista para almacenar la fecha y el usuario con más tweets
    top_dates_users = []

    for date, user_tweets in date_user_tweets.items():
        max_user = max(user_tweets.items(), key=lambda x: x[1])  # Usuario más activo

        # Agregar el resultado a la lista sin la cantidad total de tweets
        top_dates_users.append((datetime.strptime(date, '%Y-%m-%d').date(), max_user[0], date_tweet_count[date]))

    # Ordenar las fechas de forma descendente por la cantidad total de tweets
    sorted_dates = sorted(top_dates_users, key=lambda x: x[2], reverse=True)

    # Retornar solo las 10 fechas con más tweets (sin la cantidad total)
    return [(date, user) for date, user, _ in sorted_dates[:10]]


Enfoque de Memoria

In [110]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Devuelve las 10 fechas con más tweets y el usuario que más tweets publicó en esas fechas.

    """

    # Inicializa contadores de tweets por fecha y usuario
    date_tweet_count = defaultdict(int)
    date_user_tweets = defaultdict(lambda: defaultdict(int))

    # Procesar los tweets directamente del archivo
    with open(file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            tweet = json.loads(line)
            date_str = tweet["date"][:10]  # Extrae la parte de la fecha
            username = tweet["user"]["username"]  # Obtiene el nombre de usuario

            # Incrementar el contador para la fecha y el usuario
            date_tweet_count[date_str] += 1
            date_user_tweets[date_str][username] += 1

    # Lista para almacenar la fecha y el usuario con más tweets
    top_dates_users = []

    for date_str, user_tweets in date_user_tweets.items():
        # Almacena la fecha como un objeto date directamente
        current_date = datetime.strptime(date_str, '%Y-%m-%d').date()
        max_user = max(user_tweets.items(), key=lambda x: x[1])  # Usuario más activo

        # Agregar el resultado a la lista sin la cantidad total de tweets
        top_dates_users.append((current_date, max_user[0]))

    # Ordenar las fechas de forma descendente por la cantidad total de tweets
    sorted_dates = sorted(top_dates_users, key=lambda x: date_tweet_count[x[0].isoformat()], reverse=True)

    # Retornar solo las 10 fechas con más tweets (sin la cantidad total)
    return sorted_dates[:10]




Enfoque de Tiempo

In [111]:
def q1_time(file_path: str) -> List[Tuple[date, str]]:
    """
    Devuelve las 10 fechas con más tweets y el usuario que más tweets publicó en esas fechas.

    """

    # Inicializa contadores de tweets por fecha y usuario
    date_tweet_count = defaultdict(int)
    date_user_tweets = defaultdict(lambda: defaultdict(int))

    # Procesar los tweets directamente del archivo
    with open(file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            tweet = json.loads(line)
            date_str = tweet["date"][:10]  # Extrae la parte de la fecha
            username = tweet["user"]["username"]  # Obtiene el nombre de usuario

            # Incrementar el contador para la fecha y el usuario
            date_tweet_count[date_str] += 1
            date_user_tweets[date_str][username] += 1

    # Lista para almacenar la fecha y el usuario con más tweets
    top_dates_users = []

    for date_str, user_tweets in date_user_tweets.items():
        # Almacena la fecha como un objeto date directamente
        current_date = datetime.strptime(date_str, '%Y-%m-%d').date()
        max_user, _ = max(user_tweets.items(), key=lambda x: x[1])  # Usuario más activo

        # Agregar el resultado a la lista sin la cantidad total de tweets
        top_dates_users.append((current_date, max_user))

    # Ordenar las fechas de forma descendente por la cantidad total de tweets
    sorted_dates = sorted(top_dates_users, key=lambda x: date_tweet_count[x[0].isoformat()], reverse=True)

    # Retornar solo las 10 fechas con más tweets (sin la cantidad total)
    return sorted_dates[:10]




# Análisis de uso de memoria y tiempo de ejecución

In [112]:

# Medición para la versión optimizada en tiempo
result_q1_time, time_q1_time, mem_usage_time = measure_function(q1_time, file_path)
print(f"\nResultado Q1 (Tiempo): {result_q1_time}")
print(f"Tiempo de ejecución (Tiempo): {time_q1_time:.2f} segundos")
print(f"Uso máximo de memoria (Tiempo): {mem_usage_time} MiB")

# Medición para la versión optimizada en memoria
result_q1_memory, time_q1_memory, mem_usage_memory = measure_function(q1_memory, file_path)
print(f"\nResultado Q1 (Memoria): {result_q1_memory}")
print(f"Tiempo de ejecución (Memoria): {time_q1_memory:.2f} segundos")
print(f"Uso máximo de memoria (Memoria): {mem_usage_memory} MiB")

# Medición para el código deficiente
result_q1_deficiente, time_q1_deficiente, mem_usage_deficiente = measure_function(q1_deficiente, file_path)
print(f"\nResultado Q1 (Deficiente): {result_q1_deficiente}")
print(f"Tiempo de ejecución (Deficiente): {time_q1_deficiente:.2f} segundos")
print(f"Uso máximo de memoria (Deficiente): {mem_usage_deficiente} MiB")





Resultado Q1 (Tiempo): [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
Tiempo de ejecución (Tiempo): 3.82 segundos
Uso máximo de memoria (Tiempo): 1373.01953125 MiB

Resultado Q1 (Memoria): [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23)

2. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.


Código que no considera optimizar recursos de procesamiento

In [113]:
def q2_deficiente(file_path: str) -> List[Tuple[str, int]]:
    """
    Devuelve los 10 emojis más usados en el archivo JSON de tweets.
    Este método está optimizado para el uso de memoria al procesar el archivo línea por línea.
    
    """

    # Patrón regex para encontrar emojis en el texto.
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F900-\U0001F9FF]|[\u2600-\u2B55]'
    )

    # Carga todos los tweets en memoria (ineficiente)
    tweet_list = open(file_path, 'r', encoding='utf-8').readlines()
    
    # Inicializa un contador para los emojis.
    emoji_counter = Counter()

    # Procesa cada línea (tweet) del archivo.
    for line in tweet_list:
        try:
            # Convierte la línea JSON a un diccionario.
            tweet = json.loads(line)
            
            # Verifica que el tweet contenga la clave 'content'.
            if 'content' in tweet:
                # Busca todos los emojis en el contenido del tweet.
                emojis = emoji_pattern.findall(tweet["content"])
                
                # Incrementa el contador para cada emoji encontrado.
                for emoji in emojis:
                    emoji_counter[emoji] += 1
                    
        except (json.JSONDecodeError, KeyError):
            # Ignora líneas que no se pueden decodificar como JSON o que no tienen la clave 'content'.
            continue

    # Obtiene los 10 emojis más comunes y sus conteos.
    top_emojis = emoji_counter.most_common(10)

    # Retorna la lista de los 10 emojis más utilizados.
    return top_emojis



Enfoque de Memoria

In [114]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Devuelve los 10 emojis más usados en el archivo JSON de tweets.
    Este método está optimizado para el uso de memoria al procesar el archivo línea por línea.
    
    """

    # Patrón regex para encontrar emojis en el texto.
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F900-\U0001F9FF]|[\u2600-\u2B55]'
    )

    # Inicializa un contador para los emojis.
    emoji_counter = Counter()
    
    try:
        # Abre el archivo JSON para leer línea por línea.
        with open(file_path, 'r', encoding='utf-8') as json_file:
            # Procesa cada línea (tweet) del archivo.
            for line in json_file:
                try:
                    # Convierte la línea JSON a un diccionario.
                    tweet = json.loads(line)
                    
                    # Verifica que el tweet contenga la clave 'content'.
                    if 'content' in tweet:
                        # Busca todos los emojis en el contenido del tweet.
                        emojis = emoji_pattern.findall(tweet["content"])
                        
                        # Actualiza el contador con los emojis encontrados.
                        emoji_counter.update(emojis)

                except (json.JSONDecodeError, KeyError):
                    # Ignora líneas que no se pueden decodificar como JSON o que no tienen la clave 'content'.
                    continue

        # Obtiene los 10 emojis más comunes y sus conteos.
        top_emojis = emoji_counter.most_common(10)
        
        # Retorna la lista de los 10 emojis más utilizados.
        return top_emojis

    except FileNotFoundError:
        # Maneja el caso en que el archivo no se encuentra.
        print(f"Error: El archivo '{file_path}' no se encuentra.")
        return []



Enfoque de Tiempo

In [115]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Devuelve los 10 emojis más usados en el archivo JSON de tweets.
    Este método está optimizado para el tiempo de ejecución al procesar el archivo de manera eficiente.
      
    """

    # Patrón regex para encontrar emojis en el texto.
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F900-\U0001F9FF]|[\u2600-\u2B55]'
    )

    # Inicializa un contador para los emojis.
    emoji_counter = Counter()
    
    try:
        # Abre el archivo JSON para leer línea por línea.
        with open(file_path, 'r', encoding='utf-8') as json_file:
            # Procesa cada línea (tweet) del archivo.
            for line in json_file:
                try:
                    # Convierte la línea JSON a un diccionario.
                    tweet = json.loads(line)
                    
                    # Verifica que el tweet contenga la clave 'content'.
                    if 'content' in tweet:
                        # Busca todos los emojis en el contenido del tweet.
                        emojis = emoji_pattern.findall(tweet["content"])
                        
                        # Actualiza el contador con los emojis encontrados.
                        emoji_counter.update(emojis)

                except (json.JSONDecodeError, KeyError):
                    # Ignora líneas que no se pueden decodificar como JSON o que no tienen la clave 'content'.
                    continue

        # Obtiene los 10 emojis más comunes y sus conteos.
        top_emojis = emoji_counter.most_common(10)
        
        # Retorna la lista de los 10 emojis más utilizados.
        return top_emojis

    except FileNotFoundError:
        # Maneja el caso en que el archivo no se encuentra.
        print(f"Error: El archivo '{file_path}' no se encuentra.")
        return []



# Análisis de uso de memoria y tiempo de ejecución

In [116]:
# Medición para la versión optimizada en tiempo
result_q2_time, time_q2_time, mem_usage_time = measure_function(q2_time, file_path)
print(f"\nResultado Q2 (Tiempo): {result_q2_time}")
print(f"Tiempo de ejecución (Tiempo): {time_q2_time:.2f} segundos")
print(f"Uso máximo de memoria (Tiempo): {mem_usage_time} MiB")

# Medición para la versión optimizada en memoria
result_q2_memory, time_q2_memory, mem_usage_memory = measure_function(q2_memory, file_path)
print(f"\nResultado Q2 (Memoria): {result_q2_memory}")
print(f"Tiempo de ejecución (Memoria): {time_q2_memory:.2f} segundos")
print(f"Uso máximo de memoria (Memoria): {mem_usage_memory} MiB")

# Medición para el código deficiente
result_q2_deficiente, time_q2_deficiente, mem_usage_deficiente = measure_function(q2_deficiente, file_path)
print(f"\nResultado Q2 (Deficiente): {result_q2_deficiente}")
print(f"Tiempo de ejecución (Deficiente): {time_q2_deficiente:.2f} segundos")
print(f"Uso máximo de memoria (Deficiente): {mem_usage_deficiente} MiB")



Resultado Q2 (Tiempo): [('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
Tiempo de ejecución (Tiempo): 4.43 segundos
Uso máximo de memoria (Tiempo): 1356.2578125 MiB

Resultado Q2 (Memoria): [('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
Tiempo de ejecución (Memoria): 4.37 segundos
Uso máximo de memoria (Memoria): 1356.2578125 MiB

Resultado Q2 (Deficiente): [('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
Tiempo de ejecución (Deficiente): 4.57 segundos
Uso máximo de memoria (Deficiente): 1638.50390625 MiB


3. Análisis de sentimiento
Enfoque de Memoria

Código que no considera optimizar recursos de procesamiento

In [117]:
import json
import re
from collections import defaultdict
from typing import List, Tuple

def process_tweet(line: str, mention_counter: defaultdict) -> None:
    """
    Procesa un tweet para extraer menciones de usuarios de manera ineficiente.

    """
    # Cargar el tweet desde la línea JSON
    tweet = json.loads(line)

    # Obtener el nombre de usuario del autor
    user = tweet.get("user", {}).get("username")
    # Obtener el contenido del tweet
    content = tweet.get("content", "")

    if user:
        # Incrementar el contador para el autor del tweet
        mention_counter[user] += 1
        
        # Extraer menciones de otros usuarios en el contenido del tweet
        mentions = re.findall(r'@(\w+)', content)

        # Incrementar el contador de menciones por cada usuario mencionado
        for mention in mentions:
            # Incrementa el contador de menciones para cada mención
            mention_counter[mention] += 1

def q3_deficiente(file_path: str) -> List[Tuple[str, int]]:
    """
    Analiza un archivo de tweets y cuenta las menciones de usuarios de manera ineficiente.
    
    Este método abre el archivo, procesa cada tweet y retorna las 10 menciones más comunes.
    """
    # Inicializar un contador de menciones
    mention_counter = defaultdict(int)

    # Abrir el archivo JSON y procesar línea por línea
    with open(file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            process_tweet(line, mention_counter)  # Procesar cada línea

    # Ordenar los usuarios por el conteo de menciones de forma descendente
    sorted_mentions = sorted(mention_counter.items(), key=lambda x: x[1], reverse=True)

    # Devolver las 10 menciones más comunes
    return sorted_mentions[:10]


  

Enfoque de Memoria

In [118]:
import json
import re
from collections import Counter
from typing import List, Tuple

def process_tweet_memory(line: str, mention_counter: Counter) -> None:
    """
    Procesa un tweet para extraer menciones de usuarios.

    """
    try:
        # Cargar el tweet desde la línea JSON
        tweet = json.loads(line)
        # Obtener el nombre de usuario del autor
        user = tweet.get("user", {}).get("username")
        # Obtener el contenido del tweet
        content = tweet.get("content", "")

        if user:
            # Incrementar el contador para el autor del tweet
            mention_counter[user] += 1
            # Extraer menciones de otros usuarios en el contenido del tweet
            mention_counter.update(re.findall(r'@(\w+)', content))
    except json.JSONDecodeError as e:
        # Imprimir un mensaje de error si hay un problema con el formato JSON
        print(f"Error de JSON: {e}")

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Analiza un archivo de tweets y cuenta las menciones de usuarios.

    """
    mention_counter = Counter()  # Contador de menciones

    try:
        # Abrir el archivo JSON para leer línea por línea
        with open(file_path, 'r', encoding='utf-8') as json_file:
            for line in json_file:
                # Procesar cada línea y actualizar el contador
                process_tweet_memory(line, mention_counter)

        # Devolver las 10 menciones más comunes
        return mention_counter.most_common(10)

    except FileNotFoundError:
        # Imprimir un mensaje de error si el archivo no se encuentra
        print(f"Error: El archivo '{file_path}' no se encuentra.")
        return []





Enfoque de Tiempo

In [119]:
def process_tweet_time(line: str) -> defaultdict:
    """
    Procesa un tweet para extraer menciones de usuarios y devuelve un contador local.

    """
    local_counter = defaultdict(int)  # Usar un diccionario simple
    try:
        tweet = json.loads(line)
        user = tweet.get("user", {}).get("username")
        content = tweet.get("content", "")

        if user:
            local_counter[user] += 1  # Incrementa el conteo para el usuario

            # Extrae menciones de otros usuarios en el contenido
            mentions = re.findall(r'@(\w+)', content)
            for mention in mentions:
                local_counter[mention] += 1  # Incrementa el conteo para cada mención
    except json.JSONDecodeError:
        pass  # Ignora líneas que no se pueden decodificar como JSON

    return local_counter

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Analiza un archivo de tweets y cuenta las menciones de usuarios.

    """
    total_counter = defaultdict(int)  # Contador global

    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            with ThreadPoolExecutor() as executor:
                # Procesa cada línea del archivo de forma concurrente
                results = executor.map(process_tweet_time, json_file)

                # Suma los contadores locales al contador total
                for local_counter in results:
                    for user, count in local_counter.items():
                        total_counter[user] += count

        # Obtiene las 10 menciones más comunes
        return sorted(total_counter.items(), key=lambda x: x[1], reverse=True)[:10]

    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no se encuentra.")
        return []




# Análisis de uso de memoria y tiempo de ejecución

In [120]:
# Medición para la versión optimizada en tiempo
result_q3_time, time_q3_time, mem_usage_time = measure_function(q3_time, file_path)
print(f"\nResultado Q3 (Tiempo): {result_q3_time}")
print(f"Tiempo de ejecución (Tiempo): {time_q3_time:.2f} segundos")
print(f"Uso máximo de memoria (Tiempo): {mem_usage_time} MiB")

# Medición para la versión optimizada en memoria
result_q3_memory, time_q3_memory, mem_usage_memory = measure_function(q3_memory, file_path)
print(f"\nResultado Q3 (Memoria): {result_q3_memory}")
print(f"Tiempo de ejecución (Memoria): {time_q3_memory:.2f} segundos")
print(f"Uso máximo de memoria (Memoria): {mem_usage_memory} MiB")

# Medición para el código deficiente
result_q3_deficiente, time_q3_deficiente, mem_usage_deficiente = measure_function(q3_deficiente, file_path)
print(f"\nResultado Q3 (Deficiente): {result_q3_deficiente}")
print(f"Tiempo de ejecución (Deficiente): {time_q3_deficiente:.2f} segundos")
print(f"Uso máximo de memoria (Deficiente): {mem_usage_deficiente} MiB")






Resultado Q3 (Tiempo): [('narendramodi', 2261), ('Kisanektamorcha', 1837), ('RakeshTikaitBKU', 1643), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1037), ('jot__b', 1035), ('rihanna', 972), ('UNHumanRights', 962)]
Tiempo de ejecución (Tiempo): 10.83 segundos
Uso máximo de memoria (Tiempo): 1339.41796875 MiB

Resultado Q3 (Memoria): [('narendramodi', 2261), ('Kisanektamorcha', 1837), ('RakeshTikaitBKU', 1643), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1037), ('jot__b', 1035), ('rihanna', 972), ('UNHumanRights', 962)]
Tiempo de ejecución (Memoria): 4.68 segundos
Uso máximo de memoria (Memoria): 1299.19921875 MiB

Resultado Q3 (Deficiente): [('narendramodi', 2261), ('Kisanektamorcha', 1837), ('RakeshTikaitBKU', 1643), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1037), ('jot__b', 1035), ('rihanna', 972), ('UNHumanRights', 962)]
Tiempo de ejecución (Deficiente): 5.09 seg